# 1. Introduction to Pydantic
Pydantic is a powerful library that allows for data parsing and validation using Python type hints. It ensures that your data is structured, validated, and converted as needed, making it essential for projects that handle data input/output.

## Why Use Pydantic?
Provides data validation with minimal effort.
Leverages Python's type hints for better code clarity.
Automatically handles type conversion.
Offers customizable constraints on fields.

# 2. Installing Pydantic
To begin, install Pydantic using pip:

In [ ]:
%pip install pydantic

# pip install pydantic[email]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Without Type hinting:

In [ ]:
def get_full_name(first_name, last_name):
    full_name = first_name.title() + " " + last_name.title()
    return full_name


print(get_full_name("john", "doe"))

John Doe


With Type hinting:

In [ ]:
def get_full_name(first_name: str, last_name: str) -> str:
    full_name = first_name.title() + " " + last_name.title()
    return full_name


print(get_full_name("john", "doe"))


John Doe


# 3. Basic Data Validation Using Pydantic Models
Pydantic models are similar to Python data classes, but they come with built-in validation and type conversion.

Example: Basic Pydantic Model

In [ ]:
from pydantic import BaseModel

In [ ]:
class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

In [ ]:
user = User(id='123')

In [ ]:
assert user.name == 'Jane Doe'
assert user.id == 123
assert isinstance(user.id, int)

### Explanation:

The BaseModel automatically validates and converts types.
The field name has a default value 'Jane Doe'.
The id field is validated to be an integer, even if a string is passed.


# 4. Nested Models and Lists
You can also nest Pydantic models and work with lists of models.

Example: Nested Models and Lists

In [ ]:
class Group(BaseModel):
    id: int
    members: list[User] = []


In [ ]:

group = Group(id=123)

In [ ]:
group.members

[]

In [ ]:
group.members.append(user)

In [ ]:
group.members

[User(id=123, name='Jane Doe')]

### Explanation:

We define a Group model with a list of User objects.
This allows for easy nesting and working with structured data.

# 5. Self-Referential Models
Pydantic allows for self-referential models, where a model can reference itself.

### Example: Users with Friends

In [ ]:
class User(BaseModel):
    id: int
    name: str = 'Jane Doe'
    friends: list[User] = []

In [ ]:
monica = User(id=11, name='Monica')
pheebe = User(id=12, name='Pheebe')

In [ ]:
monica.friends.append(pheebe)

In [ ]:
monica.friends

[User(id=12, name='Pheebe', friends=[])]

### Explanation:

The friends field contains a list of User objects.
This creates a self-referential structure, commonly used in social networks or tree-like data.


# 6. Using Field for Customization
The Field function in Pydantic allows for advanced customization of model fields, such as default values, aliases, and constraints.

### Example: Generating UUIDs with Field

In [ ]:
from uuid import uuid4

from pydantic import BaseModel, Field

class User(BaseModel):
    id: str = Field(default_factory=lambda: uuid4().hex)
    name: str = Field(default='John Doe', alias='username')

In [ ]:
user = User(username='johndoe')

In [ ]:
user.model_dump(by_alias=True)

{'id': 'cd78f73129e24525b41f6358be48f400', 'username': 'johndoe'}

# 7. Defining Constraints on Numerical and String Fields
Pydantic allows for adding constraints to fields using the Field function. This ensures that input data adheres to the rules specified.

## 7.1 Numerical Constraints
You can add constraints such as greater than (gt), less than (lt), or values divisible by a specific number.

In [ ]:
class NumericalConstraints(BaseModel):
    positive: int = Field(gt=0)
    non_positive: int = Field(le=0)
    even: int = Field(multiple_of=2)

### Explanation:

gt=0 ensures that the number is positive.
le=0 ensures that the number is non-positive.
multiple_of=2 enforces that the number is even.

## 7.2 String Constraints
You can also set constraints for strings, such as minimum and maximum lengths or patterns using regular expressions.

In [ ]:
class StringConstraints(BaseModel):
    short: str = Field(min_length=3)
    long: str = Field(max_length=10)
    regex: str = Field(pattern=r'^\d*$')

### Explanation:

min_length=3 enforces a minimum length.
max_length=10 enforces a maximum length.
pattern=r'^\d*$' uses a regular expression to allow only digits.


# 8. Handling Validation Errors
## Validating Employee Data
In this section, we’ll expand on how to create an Employee model using Pydantic, which includes various field types such as UUIDs, Enums, and custom validators to ensure the data's validity.

In [ ]:
from datetime import date, timedelta
from uuid import UUID, uuid4
from enum import Enum
from pydantic import BaseModel, EmailStr, Field, field_validator

### Key Features in this Example:
* UUID for Unique Employee ID: Each employee is automatically assigned a unique identifier using uuid4.
* Enum for Departments: We define the available departments using an Enum, which ensures only specific values can be assigned.
* Email Validation: Ensures that only emails matching a specific domain (@example.com) are valid.
* Date Validation: Custom validation to ensure that employees are at least 18 years old.
* Salary Constraints: Ensures that the salary is a positive number.


In [ ]:
class Department(Enum):
    HR = "HR"
    SALES = "SALES"
    IT = "IT"
    ENGINEERING = "ENGINEERING"

class Employee(BaseModel):
    employee_id: UUID = Field(default_factory=uuid4, frozen=True)
    name: str = Field(min_length=1, frozen=True)
    email: EmailStr = Field(pattern=r".+@example\.com$")
    date_of_birth: date = Field(alias="birth_date", repr=False, frozen=True)
    salary: float = Field(alias="compensation", gt=0, repr=False)
    department: Department
    elected_benefits: bool

    @field_validator("date_of_birth")
    @classmethod
    def check_valid_age(cls, date_of_birth: date) -> date:
        today = date.today()
        eighteen_years_ago = date(today.year - 18, today.month, today.day)

        if date_of_birth > eighteen_years_ago:
            raise ValueError("Employees must be at least 18 years old.")

        return date_of_birth

## 8.1 Example: Employee Validation with a Custom Validator
Let’s see how the model validates data. We’ll try to create an employee who is younger than 18 years old, which should trigger the custom validator.

In [ ]:
young_employee_data = {
    "name": "John Doe",
    "email": "johndoe@example.com",
    "birth_date": date.today() - timedelta(days=365 * 17),
    "compensation": 90_000,
    "department": "SALES",
    "elected_benefits": True,
}

In [ ]:
try:
    young_employee = Employee(**young_employee_data)
except ValueError as e:
    print(f"Validation Error: {e}")

ValidationError: 1 validation error for Employee
birth_date
  Value error, Employees must be at least 18 years old. [type=value_error, input_value=datetime.date(2007, 9, 29), input_type=date]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error

In [ ]:
Employee.model_validate(young_employee_data)

ValidationError: 1 validation error for Employee
birth_date
  Value error, Employees must be at least 18 years old. [type=value_error, input_value=datetime.date(2007, 9, 29), input_type=date]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error

## 8.2 Successful Employee Creation
Now, let’s create a valid employee who is over 18 years old.



In [ ]:
valid_employee_data = {
    "name": "Jane Doe",
    "email": "janedoe@example.com",
    "birth_date": date.today() - timedelta(days=365 * 25),  # 25 years old
    "compensation": 100_000,
    "department": "HR",
    "elected_benefits": False,
}

# Creating a valid Employee instance
valid_employee = Employee(**valid_employee_data)
print(valid_employee)

# 9. Further Learning
To further explore Pydantic:

Look into custom data validation using @validator decorators.
Explore Pydantic's integration with FastAPI for building APIs with automatic data validation.
